In [53]:
import pandas as pd
from pandas import DataFrame
from pandas import read_excel
from matplotlib import pyplot as plt
import seaborn as sns
import numpy as np
import pymysql
from sqlalchemy import create_engine
import urllib

In [54]:
HOSTNAME = "localhost"
PORT = 3306
USERNAME = "root"
PASSWORD = "123qwe!@#"
DATABASE = "myschool"
CHARSET = "utf8"

In [55]:
dbcon = pymysql.connect(host = HOSTNAME, port = PORT, user = USERNAME, password = PASSWORD,
                       db = DATABASE, charset = CHARSET)

In [56]:
cursor = dbcon.cursor(pymysql.cursors.DictCursor)

# 문제 1

In [57]:
sql = """SELECT s.name, s.grade, s.birthdate, s.height, s.weight,
d.dname, p.name from student s, department d, professor p
WHERE s.deptno = d.deptno AND p.profno = s.profno
having s.grade = %s"""

for i in range(1, 5):
    cursor.execute(sql, i)
    stu = cursor.fetchall()
    df = DataFrame(stu)
    df.to_excel('%d학년' % i+'.xlsx', sheet_name = '%d학년' % i)

# 문제 2

In [58]:
sql = """SELECT avg(height) 평균_키, avg(weight) 평균_몸무게 from student
group by grade order by grade"""

cursor.execute(sql)
avkm = DataFrame(cursor.fetchall(), index = ['1학년', '2학년', '3학년', '4학년'])
avkm.to_csv('학년별 신체검사 자료.csv', encoding='euc-kr')

# 문제 3

In [59]:
HOSTNAME = "localhost"
PORT = 3306
USERNAME = "root"
PASSWORD = "123qwe!@#"
DATABASE = "myschool"
CHARSET1 = "utf8"
CHARSET2 = "utf-8"

In [60]:
con_str_fmt = "mysql+mysqldb://{0}:{1}@{2}:{3}/{4}?charset={5}"

con_str = con_str_fmt.format(USERNAME, urllib.parse.quote(PASSWORD),
                             HOSTNAME, PORT, DATABASE, CHARSET1)

In [61]:
pymysql.install_as_MySQLdb()
import MySQLdb

In [62]:
engine = create_engine(con_str, encoding = CHARSET2)
conn = engine.connect()

In [73]:
sql = """SELECT d.deptno, pro, stu from department d
LEFT OUTER JOIN(
SELECT deptno, count(deptno) as pro FROM professor group by deptno
) AS p1 ON d.deptno = p1.deptno
LEFT OUTER JOIN(
SELECT deptno , count(deptno) as stu FROM student group by deptno
) AS s1 ON d.deptno = s1.deptno 
"""

cursor.execute(sql)
res = cursor.fetchall()
df = DataFrame(res)
df

,deptno,pro,stu
0,101,4,8.0
1,102,2,4.0
2,201,1,4.0
3,202,1,NaN


In [74]:
df.to_sql('department_people', con = conn, index = False)

conn.close()
dbcon.close()